## LangChain chains

In [6]:
!pip install -q langchain langchain-openai python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


### Prosty chain: prompt → model → wynik

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś ekspertem programowania w Pythonie i geniuszem w dziedzinie AI."),
    ("user", "Napisz kod związany z tematem {topic}")
])

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm | StrOutputParser()
result = chain.invoke({'topic': 'AGI'})


In [8]:
print(result)

Tworzenie sztucznej inteligencji ogólnej (AGI) to bardzo ambitne i złożone zadanie, które wykracza poza możliwości prostego kodu. AGI odnosi się do systemów AI, które mogą wykonywać dowolne zadanie intelektualne, które potrafi wykonać człowiek. Niemniej jednak, mogę pokazać prosty przykład, który ilustruje niektóre koncepcje związane z uczeniem maszynowym i AI.

Poniżej znajduje się przykład prostego modelu uczenia maszynowego w Pythonie, który wykorzystuje bibliotekę `scikit-learn` do klasyfikacji danych. To nie jest AGI, ale może być krokiem w kierunku bardziej zaawansowanych systemów AI.

```python
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Generowanie przykładowych danych
# W tym przypadku użyjemy prostego zbioru danych o kwiatach Iris
from sklearn.datasets import load_iris

# Załaduj dane
iris = load_iris()
X = iris.data
y = iris.target

### Sequential chain: dwa modele w sekwencji

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 1) Chain: streszczenie (wejście: {text} → wyjście: str)
summary_chain = (
    ChatPromptTemplate.from_messages([
        ("system", "Streść poniższy tekst w 1–2 zdaniach po polsku."),
        ("user", "{text}")
    ])
    | llm
    | StrOutputParser()
)

# 2) Adapter: zamień str → {"text": str}, żeby podać do następnego promptu
to_dict = RunnableLambda(lambda s: {"text": s})

# 3) Chain: tłumaczenie streszczenia na francuski (wejście: {text} → wyjście: str)
translate_chain = (
    ChatPromptTemplate.from_messages([
        ("system", "Przetłumacz tekst na język francuski."),
        ("user", "{text}")
    ])
    | llm
    | StrOutputParser()
)

sequential_chain = summary_chain | to_dict | translate_chain
input_text = "LangChain umożliwia tworzenie aplikacji AI poprzez łączenie modeli, promptów i narzędzi w spójne pipeline’y."

final_translation = sequential_chain.invoke({"text": input_text})

print(final_translation)


LangChain permet de créer des applications d'IA en intégrant des modèles, des invites et des outils dans des processus organisés.


### Branching chain: jedna odpowiedź, dwa przetworzenia

In [10]:
from langchain_core.runnables import RunnableParallel

# Prompt do streszczenia
prompt_summary = ChatPromptTemplate.from_template("Streść: {text}")

# Prompt do sentymentu
prompt_sentiment = ChatPromptTemplate.from_template("Określ ton wypowiedzi: {text}")

branch_chain = RunnableParallel(
    summary=(prompt_summary | llm | StrOutputParser()),
    sentiment=(prompt_sentiment | llm | StrOutputParser())
)

text = "Jestem bardzo zadowolony z tego kursu, nauczyłem się dużo o LangChain! Teraz wiem, że LangChain umożliwia tworzenie aplikacji AI poprzez łączenie modeli, promptów i narzędzi w spójne pipeline’y."
result = branch_chain.invoke({"text": text})

print(result)


{'summary': 'Uczestnik kursu jest bardzo zadowolony z nauki o LangChain, który pozwala na tworzenie aplikacji AI poprzez integrację modeli, promptów i narzędzi w spójne pipeline’y.', 'sentiment': 'Ton wypowiedzi jest pozytywny i entuzjastyczny. Osoba wyraża zadowolenie z kursu oraz podkreśla zdobyte umiejętności i wiedzę na temat LangChain, co sugeruje, że jest zmotywowana i zaintrygowana tematem.'}
